In [1]:
import os
import spacy
import codecs
from spacy import tokenizer
import pandas as pd
from pipeline.utils import *
import re

In [2]:
from gensim.models import Phrases
from gensim.models.word2vec import LineSentence

In [3]:
nlp = spacy.load('en', disable=['ner'])
spacy.info()


    Info about spaCy

    spaCy version      2.0.11         
    Location           /opt/conda/envs/tc3/lib/python3.6/site-packages/spacy
    Platform           Linux-3.13.0-149-generic-x86_64-with-debian-jessie-sid
    Python version     3.6.5          
    Models             en_core_web_lg, en_core_web_md, en_core_web_sm, en



In [4]:
# import csv
# with codecs.open(os.path.join('data','all_the_news','articles1.csv')) as f:
#     spamreader = csv.reader(f, delimiter=',', quotechar='"')
#     for row in spamreader:
#         print(', '.join(row))

In [5]:
news = pd.read_csv(os.path.join('pipeline','data','all_the_news','articles1.csv'))
news.head()

,Unnamed: 0,id,title,publication,author,date,year,month,url,content
0,0,17283,House Republicans Fret About Winning Their Hea...,New York Times,Carl Hulse,2016-12-31,2016.0,12.0,NaN,WASHINGTON — Congressional Republicans have...
1,1,17284,Rift Between Officers and Residents as Killing...,New York Times,Benjamin Mueller and Al Baker,2017-06-19,2017.0,6.0,NaN,"After the bullet shells get counted, the blood..."
2,2,17285,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...",New York Times,Margalit Fox,2017-01-06,2017.0,1.0,NaN,"When Walt Disney’s “Bambi” opened in 1942, cri..."
3,3,17286,"Among Deaths in 2016, a Heavy Toll in Pop Musi...",New York Times,William McDonald,2017-04-10,2017.0,4.0,NaN,"Death may be the great equalizer, but it isn’t..."
4,4,17287,Kim Jong-un Says North Korea Is Preparing to T...,New York Times,Choe Sang-Hun,2017-01-02,2017.0,1.0,NaN,"SEOUL, South Korea — North Korea’s leader, ..."


In [14]:
# texts = news.head(2)['content']
def prep(text):
#     text.replace('\\n', '\n')
    return re.sub('\s+', ' ', text)

texts = news.head(1000)['content'].map(prep)
# for doc in nlp.pipe(texts, batch_size=50):
#     print(type(doc), dir(doc))
#     break
        #     [print(token.text, token.pos_, token.is_alpha, token.is_stop, token.is_space) for token in sent]

In [20]:
with codecs.open("tmp/sample.txt","w+") as f:
    
    for doc in nlp.pipe(texts, batch_size=100, n_threads=4):
        lines = ''

        for sent in doc.sents:
            s = ' '.join([token.lemma_ for token in sent if not (token.is_punct or token.is_space)])
            if(len(s.replace(' ',''))!=0): # Dunno wtf is going on here. Looks like shitty data
                f.write(s)


In [23]:
linesent = LineSentence('tmp/sample.txt')
# bg_model = Phrases(linesent)

Phrase

In [24]:
common_terms = ["of", "with", "without", "and", "or", "the", "a"]
phrases = Phrases(linesent, common_terms=common_terms)

In [30]:
bigram_sent = phrases[linesent]

/opt/conda/envs/tc3/lib/python3.6/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [36]:
for i, unigram_sentence in enumerate(linesent):
    bigram_sentence = ' '.join(phrases[unigram_sentence])
    print(bigram_sentence)
    if i > 10:
        break

/opt/conda/envs/tc3/lib/python3.6/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


washington congressional_republicans have a new fear when -PRON- come to -PRON- health_care lawsuit_against the obama_administration -PRON- may winthe incoming_trump administration could choose to no_longer defend the executive_branch against the suit which challenge the administration ’s authority to spend billion_of_dollar on health_insurance subsidy for and americans hand house_republicans a big victory on issuebut a sudden loss of the dispute subsidy could conceivably because the health_care program to implode leave million of people without access_to health_insurance before republican have prepare a replacementthat could lead to chaos in the insurance_market and spur a political backlash just as republicans gain full control of the governmentto stave off that outcome republicans could find -PRON- in the awkward position of appropriate huge sum to temporarily prop up the obama health_care law anger conservative voter who have be demand an end to the law for yearin another twist don

limit_output extension: Maximum message size of 10000 exceeded with 292136 characters

sense2vec

In [31]:
# def transform_texts(texts):
#     # Load the annotation models
# #     nlp = English()
#     # Stream texts through the models. We accumulate a buffer and release
#     # the GIL around the parser, for efficient multi-threading.
#     for doc in nlp.pipe(texts, n_threads=4):
#         # Iterate over base NPs, e.g. "all their good ideas"
#         for np in doc.noun_chunks:
#             # Only keep adjectives and nouns, e.g. "good ideas"
#             while len(np) > 1 and np[0].dep_ not in ('amod', 'compound'):
#                 np = np[1:]
#             if len(np) > 1:
#                 # Merge the tokens, e.g. good_ideas
#                 np.merge(np.root.tag_, np.text, np.root.ent_type_)
#             # Iterate over named entities
#             for ent in doc.ents:
#                 if len(ent) > 1:
#                     # Merge them into single tokens
#                     ent.merge(ent.root.tag_, ent.text, ent.label_)
#         token_strings = []
#         for token in tokens:
#             text = token.text.replace(' ', '_')
#             tag = token.ent_type_ or token.pos_
#             token_strings.append('%s|%s' % (text, tag))
#         yield ' '.join(token_strings)

In [32]:
# tt = list(transform_texts(texts))

In [35]:
nlp.vocab.length

57852

In [53]:
text = "Hi there, I think I saw a furry bear peeing over there. Help!"

enabled = nlp(news.content[0])
disabled = my_nlp(news.content[0])

get_lemma = lambda x: x.lemma_
both_lemmas = list(
    zip([get_lemma(t) for t in enabled],
       [get_lemma(t) for t in disabled])
)

both_sents = list(
    zip([s for s in enabled.sents],
       [s for s in disabled.sents])
)

for left,right in both_lemmas:
    assert(left == right)

for left,right in both_sents:
    try:
        assert(left == right)
    except:
        print('LEFT')
        print(left)
        print('RIGHT')
        print(right)

LEFT
WASHINGTON  
RIGHT
WASHINGTON  —   Congressional Republicans have a new fear when it comes to their    health care lawsuit against the Obama administration: They might win.
LEFT
—   
RIGHT
The incoming Trump administration could choose to no longer defend the executive branch against the suit, which challenges the administration’s authority to spend billions of dollars on health insurance subsidies for   and   Americans, handing House Republicans a big victory on    issues.
LEFT
Congressional Republicans have a new fear when it comes to their    health care lawsuit against the Obama administration: They might win.
RIGHT
But a sudden loss of the disputed subsidies could conceivably cause the health care program to implode, leaving millions of people without access to health insurance before Republicans have prepared a replacement.
LEFT
The incoming Trump administration could choose to no longer defend the executive branch against the suit, which challenges the administration’s auth

In [11]:
%%timeit
for parsed in [nlp(c) for c in news.content[:100]]:
    for sent in parsed.sents:
        1+1
print('yay')

yay
yay
yay
yay
yay
yay
yay
yay
18.5 s ± 1.3 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
# pipeline = ['tagger', 'sentencizer']
my_nlp = spacy.load('en', disable=['ner','parser'])
my_nlp.add_pipe(nlp.create_pipe('sentencizer'))

# disabled = nlp.disable_pipes('ner', 'parser')
# disabled.add_pipe(disabled.create_pipe('sentencizer'))

print(my_nlp.pipe_names)

['tagger', 'sbd']


In [10]:
%%timeit
for parsed in [my_nlp(c) for c in news.content[:100]]:
    for sent in parsed.sents:
        1+1
print('yay')


yay
yay
yay
yay
yay
yay
yay
yay
7.23 s ± 657 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
